# Run TARDIS workflow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from stella_to_tardis_parser import parse_stella_models_to_tardis_configs

from tardis.workflows.v_inner_solver import InnerVelocitySolverWorkflow
from tardis.io.configuration.config_reader import Configuration
from tardis.io.model import read_stella_model
from tardis.util.base import atomic_number2element_symbol


# Convert stella model to TARDIS config 

Uncomment and run this following cell if you have not yet convert the stella model into tardis config files 

In [ ]:
# STELLA_model_folder_fpath = Path.cwd() / "example_stella_explosion"

# tardis_example_config_folder_path = Path.cwd() / "TARDIS_template_configs"

# tardis_config_folder = STELLA_model_folder_fpath / "tardis_configs"

# INTERPOLATE_MASS_FRACTIONS = True  # boolean, if True then interpolate the mass fractions from MESA profile onto STELLA mass grid
# SKIP_NONHOMOLOGOUS_MODELS = (
#     True  # boolean, if True then skip non-homologous models and not save them
# )
# MAX_NONHOMOLOGOUS_SHELLS = 5  # int, if active if SKIP_NONHOMOLOGOUS_MODELS is True, the maximum number of non-homologous shells to skip
# TAU_UPPER_LIMIT = 1e3  # False or float, filter out the shells that has tau larger than this value
# TAU_LOWER_LIMIT = 1e-10  # False or float, filter out the shells that has tau lower than this value
# SHRINK_SHELL_NUMBER = False  # False or int, if int then end up with this int as total shell numbers that keep the velocity range but lower the grid resolution
# L_NUC_RATIO_UPPER_LIMIT = (
#     0.8  # default 0.8, criteria to determine if the photosphere holds, means L_nuc/L_bol <= 0.8
# )

# parse_stella_models_to_tardis_configs(
#     STELLA_model_folder_fpath,
#     tardis_example_config_folder_path,
#     tardis_config_output_folder_path=None,
#     interpolate_mass_fractions=INTERPOLATE_MASS_FRACTIONS,
#     skip_nonhomologous_models=SKIP_NONHOMOLOGOUS_MODELS,
#     max_nonhomologous_shells=MAX_NONHOMOLOGOUS_SHELLS,
#     tau_upper_limit=TAU_UPPER_LIMIT,
#     tau_lower_limit=TAU_LOWER_LIMIT,
#     shrink_shell_number=SHRINK_SHELL_NUMBER,
#     l_nuc_ratio_upper_limit=L_NUC_RATIO_UPPER_LIMIT,
# )

Setup file paths and names -- change to correct final paths in some data dir, and pick a specific day for that model

Config is output by the STELLA_to_TARDIS.ipynb notebook currently. We will want to make a pared down script for this.


In [ ]:
STELLA_model_folder_fpath = Path.cwd() / "example_stella_explosion"

chosen_day = "001"

config_fname = f"Day_{chosen_day}_mesa_stella_tardis.yml"
config_fpath = STELLA_model_folder_fpath / "tardis_configs" / config_fname
config = Configuration.from_yaml(config_fpath)

Use InnerVelocitySolverWorkflow so we don't need to define an inner boundary for an arbitrary explosion model. This moves the inner boundary in tandem with solving the plasma to get a deep enough inner boundary for a physical spectrum, but not too deep where the packets will not escape from the ejecta. 

workflow.run() is where the tardis simulation actually executes. 

In [ ]:
workflow = InnerVelocitySolverWorkflow(
    config, tau=2.0 / 3, mean_optical_depth="rosseland", csvy=True
)
workflow.run()

# Check the spectrum

In [ ]:
spectrum = workflow.spectrum_solver.spectrum_real_packets
# spectrum = workflow.spectrum_solver.spectrum_integrated

wavelength = spectrum.wavelength.value[
    ::-1
]  # in Angstrom , [::-1] to make it in increasing order in wavelength
lum_dens = spectrum.luminosity_density_lambda.value[::-1]  # in erg/s/Angstrom/cm^2

plt.plot(wavelength, lum_dens)
plt.xlabel("Wavelength [$\AA$]")
plt.ylabel("Luminosity Density [$erg/s/\AA/cm^2$]")
plt.yscale("log")
plt.xscale("log")

# Compare radiative properties 

Below are some diagnostics to see if the tardis model created from stella is similar to the stella model it was created from. Note that TARDIS only needs the outermost layers, at or nearby the optically thin region.

We start by loading in the mesa model.

In [ ]:
model_fname = f"mesa.day{chosen_day}_post_Lbol_max.data"
stella_model = read_stella_model(STELLA_model_folder_fpath / "res" / model_fname)

In [ ]:
df_model_all_columns = stella_model.data

compare_stella_cols = ["radiation_temperature", "tau", "n_e"]
compare_tardis_values = [
    workflow.simulation_state.t_radiative,
    np.exp(workflow.final_integrated_tau),
    workflow.plasma_solver.electron_densities.values[-workflow.simulation_state.no_of_shells :],
]
tardis_labels = ["t_rad", "itg_tau", "n_e"]

fig, axes = plt.subplots(
    len(compare_stella_cols), 1, figsize=(6, len(compare_stella_cols) * 3), sharex=True
)
fig.subplots_adjust(hspace=0.0)

for i, ax in enumerate(axes):
    ax.plot(
        df_model_all_columns["cell_center_v"] / 1e5,
        df_model_all_columns[compare_stella_cols[i]],
        label=f"STELLA {compare_stella_cols[i]}",
        color="tab:blue",
    )
    ax.plot(
        workflow.simulation_state.v_inner / 1e5,
        compare_tardis_values[i],
        label=f"TARDIS {tardis_labels[i]}",
        color="tab:orange",
    )
    ax.set_xlabel("Velocity (km/s)")
    ax.set_ylabel(compare_stella_cols[i])
    ax.set_yscale("log")
    ax.grid(alpha=0.3)
    ax.legend()


# Tardis visualization and diagnostic tools

In [ ]:
from tardis.visualization import SDECPlotter, LIVPlotter, LineInfoWidget, GrotrianWidget
from astropy import units as u

This cell is a hack at the time of the 2025 connector to pass an appropriate object to the SDEC and LIV plotters, when using a workflow instead of a tardis simulation object.

In [ ]:
class DummySimForPlots:
    class DummyTransport:
        def __init__(self, transport_state):
            self.transport_state = transport_state

    def __init__(self, plasma, spectrum_solver, simulation_state, transport_state):
        self.plasma = plasma
        self.spectrum_solver = spectrum_solver
        self.simulation_state = simulation_state
        self.transport = self.DummyTransport(transport_state)


In [ ]:
dummy_sim = DummySimForPlots(
    workflow.plasma_solver,
    workflow.spectrum_solver,
    workflow.simulation_state,
    workflow.transport_state,
)

In [ ]:
plotter = SDECPlotter.from_simulation(dummy_sim)

Below is a spectral decomposition plot, showing which elements contribute to absorption and emission at each wavelength. 

Note that the tardis spectrum is wider than this plot is showing by default, from 2000 to 25000 angstroms, but set in the config for the run.

In [ ]:
plotter.generate_plot_mpl(packet_wvl_range=[5000, 10000] * u.AA)

In [ ]:
livplot = LIVPlotter.from_simulation(dummy_sim)

Below is a last interaction velocity plot, showing where in the ejecta photons interact with certain elements. 

In [ ]:
livplot.generate_plot_mpl()
plt.yscale("log")

Here we check to see the mass fractions of certain elements throughout the ejecta. It will be uniform for the outer ejecta, but non-uniform when mixing reaches into the photosphere, which can happen when more of the ejecta is optically thin.

The cell immediately below the plot shows raw mass fractions per shell in a dataframe. 

In [ ]:
plt.plot(
    workflow.simulation_state.v_inner / 1e5,
    workflow.simulation_state.composition.elemental_mass_fraction.iloc[
        :, -workflow.simulation_state.no_of_shells :
    ].T,
)
plt.legend(
    [
        atomic_number2element_symbol(atomic_num)
        for atomic_num in workflow.simulation_state.composition.elemental_mass_fraction.index.values
    ]
)
plt.xlabel("Velocity (km/s)")
plt.ylabel("Mass Fraction")

plt.yscale("log")

In [ ]:
workflow.simulation_state.composition.elemental_mass_fraction.iloc[
    :, -workflow.simulation_state.no_of_shells :
]

Finally let's check some of the final thermodynamic quantities throughout the ejecta. 

In [ ]:
plt.plot(workflow.simulation_state.v_inner / 1e5, workflow.simulation_state.density)
plt.xlabel("Velocity (km/s)")
plt.ylabel("Density (g/cm^3)")

In [ ]:
plt.plot(workflow.simulation_state.v_inner / 1e5, workflow.simulation_state.t_radiative)
plt.xlabel("Velocity (km/s)")
plt.ylabel("Radiative Temperature (K)")